In [1]:
import numpy as np
from numpy import matlib
import scipy
from scipy import sparse as sp
from scipy import linalg
from scipy.sparse import csr_matrix as csr
import scipy.integrate as integrate
from scipy.special import binom as newton
from scipy.special import comb
from scipy.special import hermitenorm as hermite
from scipy.special import factorial
from itertools import permutations 
from scipy.stats import norm, gamma, uniform
from scipy.stats import norminvgauss as nig
import math
import timeit, time
import matplotlib.pyplot as plt

In [18]:
# i-th Euclidean COLUMN vector in R^n (it has 1 in position i-1 since we start counting from 0)
e_n_i = lambda n, i: csr.transpose(csr([[1.0 if k == (i-1) else 0.0 for k in range(n)]]))

# Basis COLUMN vector of monomials of degree n evaluated in x (it's a vector in R^(n+1)) (Eq. 2.3)
H_n_x = lambda n, x: csr.transpose(csr([[x**k for k in range(n+1)]]))

# Vectorization operator (Def. 3.1 of [Benth and Lavagnini (2021)])
def vec(X):
    # X: bi-dimensional array
    return X.reshape((np.product(X.shape), 1))

# Inverse-vectorization operator (Def. 3.1 of [Benth and Lavagnini (2021)])
def inv_vec(v, shape):
    # v: array or bidimensional array with one dimension = 1
    # shape: tuple (n,m)
    return v.reshape(shape)

# d-Kronecker product (Def. 4.1 of [Benth and Lavagnini (2021)])
def kron_d(X, Y, d):
    # X, Y: arrays
    # d: integer number
    if (d == 0):
        output = Y
    else:
        output = X
        if (d > 1):
            for k in range(1,d):
                output = sp.kron(output, X)
        output = sp.kron(output, Y)
    return(output)

# L-eliminating matrix E_{n,m} (Th. 3.5  of [Benth and Lavagnini (2021)])
def E_nm(n, m):
    # returns the L-eliminating matrix of order (n,m)
    output = csr((n+m-1, n*m), dtype=np.float32)
    for k in range(n+1): output += sp.kron(e_n_i(n+m-1,k), sp.kron(e_n_i(m,1).transpose(), e_n_i(n,k).transpose()))
    for k in range(2,m+1): output += sp.kron(e_n_i(n+m-1,n+k-1), sp.kron(e_n_i(m,k).transpose(), e_n_i(n,n).transpose()))
    return output

# m-th L-eliminating matrix E_n^{(m)} (Prop. 4.4  of [Benth and Lavagnini (2021)])
def mE_n(n,m):
    # returns the m-th L-eliminating matrix of order (n,m)
    if m == 0:
        return sp.diags([1]*(n+1))
    else:
        output = E_nm(n+1, n+1)
        if (m > 1):
            for k in range(2,m+1):
                output = E_nm(k*n+1, n+1).dot(sp.kron(sp.diags([1]*(n+1)), output))
        return output

def mE_n_series(n,m):
    output = [sp.diags([1]*(n+1))]
    if (m > 0):
        output = output + [E_nm(n+1, n+1)]
    if (m > 1):
        for k in range(2,m+1):
            output = output + [E_nm(k*n+1, n+1).dot(sp.kron(sp.diags([1]*(n+1)), output[k-1]))]
    return output

def mE_n_matrix(N,m):
    output = []
    for n in range(1,N+1):
        output_n = [sp.diags([1]*(n+1)), E_nm(n+1, n+1)]
        if (m > 1):
            for k in range(2,m+1):
                output_n = output_n + [E_nm(k*n+1, n+1).dot(sp.kron(sp.diags([1]*(n+1)), output_n[k-1]))]
        output += [output_n]
    return output

# Duplicating matrix D_{n,m} (Th. 3.9  of [Benth and Lavagnini (2021)])
def D_nm(n,m):
    # returns the L-duplicating matrix of order (n,m)
    output = csr((n*m, n+m-1), dtype=np.float32)
    for i in range(n+1):
        for j in range(m+1):
            output += sp.kron(e_n_i(n+m-1, i+j-1).transpose(),sp.kron(e_n_i(m,j), e_n_i(n,i)))
    return output

# m-th duplicating matrix D_n^{(m)} (Prop. 4.5  of [Benth and Lavagnini (2021)])
def mD_n(n,m):
    # returns the m-th L-duplicating matrix of order (n,m)
    if m == 0:
        return sp.diags([1]*(n+1))
    else:
        output = D_nm(n+1, n+1)
        if (m > 1):
            for k in range(2,m+1):
                output = sp.kron(sp.diags([1]*(n+1)), output).dot(D_nm(k*n+1, n+1))
        return output
    
def mD_n_series(n,m):
    output = [sp.diags([1]*(n+1))]
    if (m > 0):
        output = output + [D_nm(n+1, n+1)]
    if (m > 1):
        for k in range(2,m+1):
            output = output + [sp.kron(sp.diags([1]*(n+1)), output[k-1]).dot(D_nm(k*n+1, n+1))]
    return output

def mD_n_matrix(N,m):
    output = []
    for n in range(1,N+1):
        output_n = [sp.diags([1]*(n+1)), D_nm(n+1, n+1)]
        if (m > 1):
            for k in range(2,m+1):
                output_n = output_n + [sp.kron(sp.diags([1]*(n+1)), output_n[k-1]).dot(D_nm(k*n+1, n+1))]
        output += [output_n]
    return output

In [1]:
class poly_process_sparse(object):
    # This class defines a univariate polynomial diffusion process starting from his coefficients:
    # b(x) = b_0 + b_1x
    # \sigma^2(x) = \sigma_0 + \sigma_1 x + \sigma_2 x^2
    
    def __init__(self, coeff, nig_coeff = 0 ):
        # coeff: array of 5 elements (b_0, b_1, \sigma_0, \sigma_1, \sigma_2)
        self.coeff = coeff
        
    def generator_matrix(self, n):
        # creates the generator matrix by the recursion formula (Th. 5.1 in [Benth and Lavagnini (2021)])
        coeff = self.coeff
        G_n = csr([[0, 0],[coeff[0], coeff[1]]]) #G_1
        if (n > 1):
            for k in range(2,n+1):
                a_k = sp.vstack([csr((k-2, 1)), csr([[1/2*k*(k-1)*coeff[2]], [k*coeff[0]+1/2*k*(k-1)*coeff[3]]])])
                c_k = sp.vstack((csr((k, 1)), csr([[k*coeff[1]+1/2*k*(k-1)*coeff[4]]])))
                G_n = sp.hstack([sp.vstack([G_n, csr.transpose(a_k)]), c_k])
        return G_n
    
    def exp_generator_matrix(self, n, t=1):
        # creates the matrix exponential of the generator matrix by the recursion formula 
        # (Th. 5.4 in [Benth and Lavagnini (2021)])
        coeff = self.coeff
        G_n = self.generator_matrix(n)
        expG_n = csr([[1, 0],[coeff[0]/coeff[1]*(np.exp(coeff[1]*t)-1), np.exp(coeff[1]*t)]]) #exp(G_1*t)
        for k in range(2,n+1):
            a_k = sp.vstack([csr((k-2, 1)), csr([[1/2*k*(k-1)*coeff[2]], [k*coeff[0]+1/2*k*(k-1)*coeff[3]]])]).reshape((1,k))
            c_k = k*coeff[1]+1/2*k*(k-1)*coeff[4]
            inv_Lambda_k = sp.linalg.inv((sp.diags([c_k]*k)-csr(G_n.toarray()[:k,:k])).tocsc()).tocsr()
            a_k_new = a_k.dot(inv_Lambda_k).dot(sp.diags([np.exp(c_k*t)]*k) - expG_n)   #expGt_list[k-2])
            expG_n = sp.hstack([sp.vstack([expG_n, a_k_new]), sp.vstack([csr((k, 1)), np.exp(c_k*t)])])
        return expG_n
      
    def exp_generator_matrix_scipy(self, n, t=1):
        # creates the matrix exponential of the generator matrix by the sp.linalg.expm function
        G_n = self.generator_matrix(n)
        expG_n = sp.linalg.expm((G_n.tocsc())*t).tocsr()
        return expG_n
                
    def simulation_path_OU(self, initial_state, t, T, N_point):
        # simulates the path of an OU process between t and T by using N_point  of points
        # initial_state: real, starting point of the process
        # t, T: positive real with t < T
        # N_point: positive integer
        # NOTICE: the OU is obtained with \sigma_1=\sigma_2 = 0
        b0 = self.coeff[0]
        b1 = self.coeff[1]
        s0 = self.coeff[2]
        time_sequence = np.linspace(t, T, N_point+1)
        delta = (T-t)/N_point
        std_BM = np.sqrt(s0*(np.exp(2*b1*delta)-1)/2/b1)
    
        Y = np.zeros(N_point+1)
        Y[0] = initial_state
        for k in np.arange(N_point):
            Y[k+1] = Y[k]*np.exp(b1*delta)+b0/b1*(np.exp(b1*delta)-1) + np.random.normal(loc = 0, scale =  std_BM)
        return Y
    
    def simulation_path_GBM(self, initial_state, t, T, N_point):
        # simulates the path of a geometric BM process between t and T by using N_point  of points
        # initial_state: real, starting point of the process
        # t, T: positive real with t < T
        # N_point: positive integer
        # NOTICE: the GBM is obtained with b_0 = \sigma_0 = \sigma_1 = 0
        b1 = self.coeff[1]
        s2 = self.coeff[4]
        delta_sequence = (T-t)/N_point
        
        Y = np.exp((b1-s2/2)*delta_sequence + np.sqrt(s2)*np.random.normal(0, np.sqrt(delta_sequence), size=(1, N_point)).T)
        Y = np.vstack([1, Y])
        Y = initial_state * Y.cumprod()
        return Y
     
    def plot_poly_process_OU(self, initial_state, t, T, N_point):
        # plots the OU path
        Y = self.simulation_path_OU(initial_state, t, T, N_point)
        time_sequence = np.linspace(t, T, N_point+1)
        return plt.plot(time_sequence, Y)

    def plot_poly_process_GBM(self, initial_state, t, T, N_point):
        # plots the GBM path
        Y = self.simulation_path_GBM(initial_state, t, T, N_point)
        time_sequence = np.linspace(t, T, N_point+1)
        return plt.plot(time_sequence, Y)
    
    def correlator_formula(self, n, m, time_sequence, coeff_vectors_list, initial_state, t=0):
        # This function computes the correlator formula (Th. 4.8 [Benth and Lavagnini (2021)])
        # n: integer, maximum power
        # m: number of polynomials - 1
        # time_sequence: (m+1)-long array with time points t < s_0 < s_1 < ... < s_m = T
        # coeff_vector_list: list of arrays; each array has lenght n+1 and is the vector of coefficients
        #                        of a polynomial
        # initial_state: : real, starting point of the process
        
        # This first version computes itself the exponential of the generator matrix and the eliminiting
        # and duplicating matrices needed
        
        #expG_n = lambda t: self.exp_generator_matrix(n = n*(m+1), t = t)
        expG_n = lambda t: self.exp_generator_matrix_scipy(n = n*(m+1), t = t)
        
        H_n_t = H_n_x(n, initial_state)
        HH_n_t = kron_d(H_n_t.transpose(), H_n_t, m)
        D = mD_n(n, m)
        E = mE_n(n, m)
        s_k = time_sequence[0]-t
        expDGE = D.dot(expG_n(s_k).dot(E))
        prod = inv_vec(expDGE.dot(vec(HH_n_t)), shape = HH_n_t.shape)
        if m>0:
            for k in range(1, m+1):
                D = mD_n(n, m-k)
                E = mE_n(n, m-k)
                s_k = time_sequence[k]-time_sequence[k-1]
                expDGE = D.dot(csr(expG_n(s_k).toarray()[:(n*(m-k+1)+1),:(n*(m-k+1)+1)]).dot(E))
                prod = prod.dot(csr.transpose(expDGE).dot(kron_d(sp.diags([1]*(n+1)), coeff_vectors_list[m-k].tocsr(), m-k)))
        return float(csr.transpose(coeff_vectors_list[m].tocsr()).dot(prod)[0,0])
    
    def correlator_formula_2(self, expG_n, n, m, time_sequence, coeff_vectors_list, initial_state, t=0):
        # This function computes the correlator formula (Th. 4.8 [Benth and Lavagnini (2021)])
        # expG_n: sparse matrix, generator matrix of order n*(m+1)
        # n: integer, maximum power
        # m: number of polynomials - 1
        # time_sequence: (m+1)-long array with time points t < s_0 < s_1 < ... < s_m = T
        # coeff_vector_list: list of arrays; each array has lenght n+1 and is the vector of coefficients
        #                        of a polynomial
        # initial_state: : real, starting point of the process
        
        # This second version needs the exponential of the generator matrix but computes itself the eliminiting
        # and duplicating matrices needed

        H_n_t = H_n_x(n, initial_state)
        HH_n_t = kron_d(H_n_t.transpose(), H_n_t, m)
        D = mD_n(n, m)
        E = mE_n(n, m)
        s_k = time_sequence[0]-t
        expDGE = D.dot(expG_n(s_k).dot(E))
        prod = csr(inv_vec(expDGE.dot(vec(HH_n_t)), shape = HH_n_t.shape))
        if m>0:
            for k in range(1, m+1):
                D = mD_n(n, m-k)
                E = mE_n(n, m-k)
                s_k = time_sequence[k]-time_sequence[k-1]
                expDGE = D.dot(csr(expG_n(s_k).toarray()[:(n*(m-k+1)+1),:(n*(m-k+1)+1)]).dot(E))
                prod = prod.dot(csr.transpose(expDGE).dot(kron_d(sp.diags([1]*(n+1)), coeff_vectors_list[m-k].tocsr(), m-k)))
        return float(csr.transpose(coeff_vectors_list[m].tocsr()).dot(prod).toarray())

    def correlator_formula_3(self, expG_n, E_list, D_list, n, m, time_sequence, coeff_vectors_list, initial_state, t=0):
        # This function computes the correlator formula (Th. 4.8 [Benth and Lavagnini (2021)])
        # expG_n: sparse matrix, generator matrix of order n*(m+1)
        # E_list: list of eliminating matrices in increasing order
        # D_list: list of duplicating matrices in increasing order
        # n: integer, maximum power
        # m: number of polynomials - 1
        # time_sequence: (m+1)-long array with time points t < s_0 < s_1 < ... < s_m = T
        # coeff_vector_list: list of arrays; each array has lenght n+1 and is the vector of coefficients
        #                        of a polynomial
        # initial_state: : real, starting point of the process
        
        # This third version needs both the exponential of the generator matrix and the eliminiting
        # and duplicating matrices

        H_n_t = H_n_x(n, initial_state)
        HH_n_t = kron_d(H_n_t.transpose(), H_n_t, m)
        D = D_list[-1]
        E = E_list[-1]
        s_k = time_sequence[0]-t
        expDGE = D.dot(expG_n(s_k).dot(E))
        prod = inv_vec(expDGE.dot(vec(HH_n_t)), shape = HH_n_t.shape)
        if m>0:
            for k in range(1, m+1):
                D = D_list[-k-1]
                E = E_list[-k-1]
                s_k = time_sequence[k]-time_sequence[k-1]
                expDGE = D.dot(csr(expG_n(s_k).toarray()[:(n*(m-k+1)+1),:(n*(m-k+1)+1)]).dot(E))
                prod = prod.dot(csr.transpose(expDGE).dot(kron_d(sp.diags([1]*(n+1)), coeff_vectors_list[m-k], m-k)))
        return float(csr.transpose(coeff_vectors_list[m].tocsr()).dot(prod)[0,0])
    
    def recursive_moment_formula(self, m, n, time_sequence, coeff_vectors_list, t):    
        # This function computes the correlators by iteratively applying the moment formula 
        # as described in [Benth and Lavagnini (2021)]
        
        # expG_n: sparse matrix, generator matrix of order n*(m+1)
        # n: integer, maximum power
        # m: number of polynomials - 1
        # time_sequence: (m+1)-long array with time points t < s_0 < s_1 < ... < s_m = T
        # coeff_vector_list: list of arrays; each array has lenght n+1 and is the vector of coefficients
        #                        of a polynomial
        # initial_state: : real, starting point of the process

        if (m==0):
            return csr.transpose(coeff_vectors_list[0].tocsr()).dot(self.exp_generator_matrix_scipy(n=n[0], t=time_sequence[0]-t))
        else:
            q_0 = csr.transpose(coeff_vectors_list[0].tocsr()).dot(self.exp_generator_matrix_scipy(n=n[0], t=time_sequence[-1]-time_sequence[-2]))
            p_1_tilde = csr((n[0]+n[1]+1,1), dtype=np.float32)
            for k in range(n[0]+1):
                for i in range(n[1]+1):
                    p_1_tilde[k+i] += q_0.toarray()[0][k]*coeff_vectors_list[1].toarray()[i]
            if (m==1):
                return self.recursive_moment_formula(0, [n[0]+n[1]], [time_sequence[0]], [p_1_tilde], t=t)
            else:
                return self.recursive_moment_formula(m-1, [n[0]+n[1],*n[2:]], time_sequence[:-1], [p_1_tilde,*coeff_vectors_list[2:]], t=t)        

In [5]:
class poly_process_sparse_levy(poly_process_sparse):
    # This class defines a univariate polynomial jump-diffusion process with coefficients
    # b(x) = b_0 + b_1x
    # \sigma^2(x) = \sigma_0 + \sigma_1 x + \sigma_2 x^2
    # and jump component driven by a NIG measure with parameters (\alpha, \beta, \mu, \delta)

    
    def __init__(self, coeff, nig_coeff = 0 ):
        # coeff: array of 5 elements (b_0, b_1, \sigma_0, \sigma_1, \sigma_2)
        # nig_coeff: array of 4 elements (\alpha, \beta, \mu, \delta)
        poly_process_sparse.__init__(self, coeff)
        self.nig_coeff = nig_coeff
        
    def generator_matrix(self, n):
        # creates the generator matrix by the recursion formula (Th. 5.1 in [Benth and Lavagnini (2021)])
        b0 = self.coeff[0]
        b1 = self.coeff[1]
        s0 = self.coeff[2]
        s1 = self.coeff[3]
        s2 = self.coeff[4]
        aa = self.nig_coeff[0]
        bb =  self.nig_coeff[1]
        mu = self.nig_coeff[2]
        delta = self.nig_coeff[3]
        G_n = csr([[0, 0],[b0, b1]]) #G_1
        if (n > 1):
            M = np.zeros(n+1)
            for k in range(2, n+1):
                M[k] = integrate.quad(lambda z: z**k*nig.pdf(z, aa, bb, loc=mu, scale=delta), a = -100, b = 100)[0]
            for k in range(2,n+1):
                if (k > 2):
                    a_levy_k = np.zeros(k-1)
                    for j in range(k-1):
                        a_levy_k[j] = newton(k, k-j)*M[k-j]
                    a_levy_k[k-2] += 1/2*k*(k-1)*s0
                    a_levy_k = csr(a_levy_k)
                else: a_levy_k = csr([newton(k, 2)*M[2] + 1/2*k*(k-1)*s0])
                
                a_k = sp.hstack([a_levy_k,  csr([k*b0+1/2*k*(k-1)*s1])])
                c_k = sp.vstack((csr((k, 1)), csr([[k*b1+1/2*k*(k-1)*s2]])))
                G_n = sp.hstack([sp.vstack([G_n, a_k]), c_k])
        return G_n
      
    def exp_generator_matrix_scipy(self, n, t=1):
        # creates the matrix exponential of the generator matrix by the sp.linalg.expm function
        G_n = self.generator_matrix(n)
        expG_n = sp.linalg.expm((G_n.tocsc())*t).tocsr()
        return expG_n
                    
    def correlator_formula(self, n, m, time_sequence, coeff_vectors_list, initial_state, t=0):
        # This function computes the correlator formula (Th. 4.8 [Benth and Lavagnini (2021)])
        # n: integer, maximum power
        # m: number of polynomials - 1
        # time_sequence: (m+1)-long array with time points t < s_0 < s_1 < ... < s_m = T
        # coeff_vector_list: list of arrays; each array has lenght n+1 and is the vector of coefficients
        #                        of a polynomial
        # initial_state: : real, starting point of the process
        
        # This first version computes itself the exponential of the generator matrix and the eliminiting
        # and duplicating matrices needed
        
        expG_n = lambda t: self.exp_generator_matrix_scipy(n = n*(m+1), t = t)
        H_n_t = H_n_x(n, initial_state)
        HH_n_t = kron_d(H_n_t.transpose(), H_n_t, m)
        D = mD_n(n, m)
        E = mE_n(n, m)
        s_k = time_sequence[0]-t
        expDGE = D.dot(expG_n(s_k).dot(E))
        prod = inv_vec(expDGE.dot(vec(HH_n_t)), shape = HH_n_t.shape)
        if m>0:
            for k in range(1, m+1):
                D = mD_n(n, m-k)
                E = mE_n(n, m-k)
                s_k = time_sequence[k]-time_sequence[k-1]
                expDGE = D.dot(csr(expG_n(s_k).toarray()[:(n*(m-k+1)+1),:(n*(m-k+1)+1)]).dot(E))
                prod = prod.dot(csr.transpose(expDGE).dot(kron_d(sp.diags([1]*(n+1)), coeff_vectors_list[m-k].tocsr(), m-k)))
        return float(csr.transpose(coeff_vectors_list[m].tocsr()).dot(prod)[0,0])
    
    def correlator_formula_2(self, expG_n, n, m, time_sequence, coeff_vectors_list, initial_state, t=0):
        # This function computes the correlator formula (Th. 4.8 [Benth and Lavagnini (2021)])
        # expG_n: sparse matrix, generator matrix of order n*(m+1)
        # n: integer, maximum power
        # m: number of polynomials - 1
        # time_sequence: (m+1)-long array with time points t < s_0 < s_1 < ... < s_m = T
        # coeff_vector_list: list of arrays; each array has lenght n+1 and is the vector of coefficients
        #                        of a polynomial
        # initial_state: : real, starting point of the process
        
        # This second version needs the exponential of the generator matrix but computes itself the eliminiting
        # and duplicating matrices needed

        H_n_t = H_n_x(n, initial_state)
        HH_n_t = kron_d(H_n_t.transpose(), H_n_t, m)
        D = mD_n(n, m)
        E = mE_n(n, m)
        s_k = time_sequence[0]-t
        expDGE = D.dot(expG_n(s_k).dot(E))
        prod = csr(inv_vec(expDGE.dot(vec(HH_n_t)), shape = HH_n_t.shape))
        if m>0:
            for k in range(1, m+1):
                D = mD_n(n, m-k)
                E = mE_n(n, m-k)
                s_k = time_sequence[k]-time_sequence[k-1]
                expDGE = D.dot(csr(expG_n(s_k).toarray()[:(n*(m-k+1)+1),:(n*(m-k+1)+1)]).dot(E))
                prod = prod.dot(csr.transpose(expDGE).dot(kron_d(sp.diags([1]*(n+1)), coeff_vectors_list[m-k].tocsr(), m-k)))
        return float(csr.transpose(coeff_vectors_list[m].tocsr()).dot(prod).toarray())

    def correlator_formula_3(self, expG_n, E_list, D_list, n, m, time_sequence, coeff_vectors_list, initial_state, t=0):
        # This function computes the correlator formula (Th. 4.8 [Benth and Lavagnini (2021)])
        # expG_n: sparse matrix, generator matrix of order n*(m+1)
        # E_list: list of eliminating matrices in increasing order
        # D_list: list of duplicating matrices in increasing order
        # n: integer, maximum power
        # m: number of polynomials - 1
        # time_sequence: (m+1)-long array with time points t < s_0 < s_1 < ... < s_m = T
        # coeff_vector_list: list of arrays; each array has lenght n+1 and is the vector of coefficients
        #                        of a polynomial
        # initial_state: : real, starting point of the process
        
        # This third version needs both the exponential of the generator matrix and the eliminiting
        # and duplicating matrices

        H_n_t = H_n_x(n, initial_state)
        HH_n_t = kron_d(H_n_t.transpose(), H_n_t, m)
        D = D_list[-1]
        E = E_list[-1]
        s_k = time_sequence[0]-t
        expDGE = D.dot(expG_n(s_k).dot(E))
        prod = inv_vec(expDGE.dot(vec(HH_n_t)), shape = HH_n_t.shape)
        if m>0:
            for k in range(1, m+1):
                D = D_list[-k-1]
                E = E_list[-k-1]
                s_k = time_sequence[k]-time_sequence[k-1]
                expDGE = D.dot(csr(expG_n(s_k).toarray()[:(n*(m-k+1)+1),:(n*(m-k+1)+1)]).dot(E))
                prod = prod.dot(csr.transpose(expDGE).dot(kron_d(sp.diags([1]*(n+1)), coeff_vectors_list[m-k].tocsr(), m-k)))
        return float(csr.transpose(coeff_vectors_list[m].tocsr()).dot(prod).toarray())

In [5]:
### OTHER AUXILIARY FUNCTIONS

def multi_index(m, N):
    # calculates all the possible combinations of {0,1,...,m+1} whose sum equals N without repetition
    # m+1 is the number of time points
    # N is the power, hence the sum of the elements of each multi index
    v = np.matlib.repmat(np.arange(N+1), 1, m+1)[0]
    perm = permutations(v, m+1) 
    l = []
    for i in list(perm): 
        if sum(i) == N:
            l = l + [i]
    l = np.ascontiguousarray(l)
    unique_l = np.unique(l.view([('', l.dtype)]*l.shape[1]))
    return unique_l.view(l.dtype).reshape((unique_l.shape[0], l.shape[1]))

def multi_coeff(v):
    # v is a multi index from milti_index(m,N)
    # calculates the multinomial coefficients appearing in the Hermite pricing series (Eq. (4.4))
    coeff = 1
    for f in v:
        coeff = coeff*math.factorial(f)
    n = np.sum(v)
    return float(math.factorial(n)/coeff)


def coeff_vectors_list(v, N):
    # v is a multi index from milti_index(m,N)
    # creates a list of euclidean basis vectors in \R^{N+1} corresponding to each element in v
    # these are the vectors of coefficients for the powers included in v
    # e.g is v = [1,2,0] and N=3, then vector_list will be [e_n_i(4, 2), e_n_i(4, 3), e_n_i(4, 1)]
    vectors_list = []
    for f in v:
        vectors_list = vectors_list + [e_n_i(N+1, f+1)]
    return vectors_list


def coeff_vectors_list2(v):
    # v is a multi index
    vectors_list = []
    for f in v:
        vectors_list = vectors_list + [e_n_i(f+1, f+1)]
    return vectors_list


def meanX(m, time_sequence):
    # Calculates the mean of a sum of OU as in Eq. (5.8)
    
    mean_sum = 0
    for j in range(m+1):
        sj = time_sequence[j]
        mean_sum += initial_state*np.exp(b1*(sj-t)) + b0*(np.exp(b1*(sj-t))-1)/b1
    return mean_sum/(m+1)


def varX(m, b1, sigma0, time_sequence, t=0):
    # Calculates the variance of a sum of OU as in Eq. (5.8)
    
    ts = np.append(t, time_sequence)
    var_sum = 0
    for j in range(m+1):
        for k1 in range(j, m+1):
            for k2 in range(j, m+1):
                var_sum += np.exp(b1*(ts[k1+1]+ts[k2+1]-2*ts[j]))-np.exp(b1*(ts[k1+1]+ts[k2+1]-2*ts[j+1]))
    return np.sqrt(sigma0*var_sum/(2*b1*(m+1)**2))

# Calculates the moments of a BM 
# (in this case no need of the moment formula for polynomial processes)
def moment_BM(p, sigma):
    # p integer: power
    # sigma: std of the BM
    if p%2 == 0:
        k = p/2
        return sigma**p*math.factorial(p)/(2**k)/math.factorial(k)
    else:
        return 0